# Spark UI 접속을 위한 docker run 명령어
docker run -p 8888:8888 -p 4040:4040 -e JUPYTER_ENABLE_LAB=yes -e GRANT_SUDO=yes --user root -v /Users/seokil/docker-vols/pyspark:/home/jovyan/work --restart always jupyter/all-spark-notebook

이경우 spark_session을 master("local") 로 생성해야 함 (아래 sample code 참조)

# RDD 연산 - 1
- Transformation 과 Action 으로 구분됨
    - Transformation : RDD는 immutable (불변) 특성을 가짐.  변겨을 위해서는 새로운 RDD를 생성하고 변경
    - Action : RDD의 내용을 조회, 저장
- Sparse 연산 : RDD의 개별 레코드를 변경할 수 없음. 모든 레코드에 대해서 동일하게 적용되는 연산. 리니지를 단순화하기 위해. 
- Lazy Evaluation 또는 Lazy Execution : 
    - Transformation 연산은 바로 실행되지 않고, parsing 만 됨. 
    - Action이 수행될때 Action에 필요한 RDD를 만들기 위한 Transformation 이 수행됨. 
    - Driver는 Action 연산을 실행하기 위해 RDD 생성 DAG를 만들고 최적화 <- 작업양 최소화 및 Shuffling시 데이터 이동 최소화


In [3]:
from pyspark.sql import SparkSession
spark_session = SparkSession.builder.master("local").appName("rdd-op-test1").getOrCreate()

In [2]:
rdd1 = spark_session.sparkContext.parallelize([0,1,2,3,4,5,6,7,8,9])
rdd2 = rdd1.filter(lambda x: x%2)

In [3]:
rdd2.collect()

[1, 3, 5, 7, 9]

# RDD 지속성 및 재사용
    - RDD는 익스큐터(executor)의 메모리에 생성됨
    - 생성된 RDD는 더아상 참조되지 않으면 (사용되지 않으면) 삭제됨.
    - 이후 Action연산이 수행되기 위해 다시 RDD를 생성해야 함. 
    - Persist()를 이용하면 메모리에 지속됨

In [4]:
prdd1 = spark_session.sparkContext.parallelize([0,1,2,3,4,5,6,7,8,9])
prdd2 = rdd1.filter(lambda x: x%2)
prdd2.persist()

PythonRDD[3] at RDD at PythonRDD.scala:53

In [5]:
rdd2.collect()

[1, 3, 5, 7, 9]

# RDD의 유형
- PairRDD : Key-Value RDD
- DoubleRDD : Double Value RDD
- DataFrame : Table 형식의 데이터 RDD
- SecuenceFileRDD
- HadoopRDD
- NewHadoopRDD
- CoGroupRDD
- JdbcRDD
- PartitioningPruningRDD :특정 파티션에 대해서는 작업이 수행되지 않도록 제한 할 수 있는 RDD
- ShuffledRDD : Shuffle 을 통해 만들어진 RDD
- UnionRDD

# 기본 RDD Transportation 연산

- RDD.map(function, preservesPartitioning=False)
- RDD.flatMap(function, preservesPartitioning=False)
- RDD.filter(function)
- RDD.distinct(numPartitions=None)
- RDD.groupBy(function, numPartitions=None)


In [4]:
shakespeare = spark_session.sparkContext.textFile("./data/shakespeare.txt")
shakespeare.take(5)

["A MIDSUMMER-NIGHT'S DREAM",
 '',
 'Now , fair Hippolyta , our nuptial hour ',
 'Draws on apace : four happy days bring in ',
 'Another moon ; but O ! methinks how slow ']

In [7]:
words = shakespeare.flatMap(lambda x: x.split(' '))
words.take(5)

['A', "MIDSUMMER-NIGHT'S", 'DREAM', '', 'Now']

In [8]:
lowercase = words.map(lambda x: x.lower())
lowercase.take(10)

['a',
 "midsummer-night's",
 'dream',
 '',
 'now',
 ',',
 'fair',
 'hippolyta',
 ',',
 'our']

In [10]:
longwords = lowercase.filter(lambda x: len(x) > 2)
longwords.take(5)

["midsummer-night's", 'dream', 'now', 'fair', 'hippolyta']

In [10]:
longwords_cnt = longwords.count()
longwords_cnt

632856

In [11]:
distinctwords = longwords.distinct()
distinctwords_cnt = distinctwords.count()
distinctwords_cnt

28734

In [16]:
groupby_firstletter = distinctwords.groupBy(lambda x: x[0].lower())
print(groupby_firstletter.take(10))
groupby_firstletter.count()

[('m', <pyspark.resultiterable.ResultIterable object at 0x7f6e216ced50>), ('d', <pyspark.resultiterable.ResultIterable object at 0x7f6e218c66d0>), ('n', <pyspark.resultiterable.ResultIterable object at 0x7f6e216f3f10>), ('f', <pyspark.resultiterable.ResultIterable object at 0x7f6e216ced90>), ('h', <pyspark.resultiterable.ResultIterable object at 0x7f6e21716110>), ('o', <pyspark.resultiterable.ResultIterable object at 0x7f6e217280d0>), ('a', <pyspark.resultiterable.ResultIterable object at 0x7f6e21733b90>), ('b', <pyspark.resultiterable.ResultIterable object at 0x7f6e216f3ed0>), ('s', <pyspark.resultiterable.ResultIterable object at 0x7f6e217672d0>), ('t', <pyspark.resultiterable.ResultIterable object at 0x7f6e21798390>)]


35

In [13]:
groupby_firstletter.take(3)

[('m', <pyspark.resultiterable.ResultIterable at 0x7f5696f5e5f0>),
 ('d', <pyspark.resultiterable.ResultIterable at 0x7f5696f5e1a0>),
 ('n', <pyspark.resultiterable.ResultIterable at 0x7f5696f5e530>)]

In [14]:
distinctwords_sort = distinctwords.sortBy(lambda x: x, ascending=False)
distinctwords_sort.take(10)

['zwaggered',
 'zur',
 'zounds',
 'zone',
 'zodiacs',
 'zodiac',
 'zephyrs',
 'zenith',
 'zenelophon',
 'zed']

In [15]:
distinctwords_sort = distinctwords.sortBy(lambda x: x[1], ascending=False)
distinctwords_sort.take(10)

['azure',
 "azur'd",
 "'zounds",
 'lysander',
 'eyes',
 'aye',
 'hymns',
 'eye',
 'sympathy',
 'myself']

In [16]:
#distinctwords.collect()
distinctwords.take(10)

["midsummer-night's",
 'dream',
 'now',
 'fair',
 'hippolyta',
 'our',
 'nuptial',
 'hour',
 'draws',
 'apace']

In [17]:
distinctwords.top(10)

['zwaggered',
 'zur',
 'zounds',
 'zone',
 'zodiacs',
 'zodiac',
 'zephyrs',
 'zenith',
 'zenelophon',
 'zed']

In [18]:
distinctwords.first()

"midsummer-night's"

# reduce vs fold
- RDD의 요소를 줄이는 액션
- fold는 초기 값으로 0를 입력

In [28]:
numbers = spark_session.sparkContext.parallelize([0,1,2,3,4,5,6,7,8,9])
numbers.reduce(lambda x, y: x+y)

45

In [20]:
numbers.fold(0, lambda x, y: x+y)

45

In [21]:
empty = spark_session.sparkContext.parallelize([])
empty.reduce(lambda x, y: x + y)

ValueError: Can not reduce() empty RDD

In [ ]:
result = empty.fold(0, lambda x, y: x + y)
result

# foreach 
RDD.foreach(function)
- Action 연산이면서 입력한 function 을 RDD 모든 엘리먼트에 적용함
- 각 worker가 함수를 실행함

In [17]:
def print_func(x):
    print(x)

longwords.foreach(print_func)